# Diferenciación automática (o algorítmica): Parte 1

### NOTA

Este notebook se iniciará en la clase y **debe** hacerse en equipo de dos personas (máximo tres). La resolución completa de los ejercicios debe ser enviada como "Tarea4.ipynb".

## Motivación

En el [notebook anterior](https://github.com/lbenet/2016-2_TSFisicaComputacional/blob/master/notas_clase/06_Derivacion_numerica.ipynb), vimos que hay diferentes formas de implementar la derivación de una función $f(x)$ en un punto $x_0$ y que éstas dependen de un parámetro $h$, que es la separación entre puntos cercanos. 

Vimos que el error absoluto en términos de $h$, cuando $h\to 0$, tiene un comportamiento distinto, y que de hecho puede ser "contaminado" por errores de numéricos.

Se obtuvo que:

- La "derivada derecha" converge *linealmente* al $h\to 0$ al valor correcto de la derivada, es decir, el error es proporcional a $\mathcal{O}(h)$. Sin embargo, para valores *suficientemente* pequeños de $h$, el valor obtenido de la derivada deja de tener sentido ya que pierde exactitud.

- La "derivada simétrica" converge *cuadráticamente*, es decir, el error escala como $\mathcal{O}(h^2)$. Al igual que la derivada derecha, para $h$ suficientemente pequeña errores "de cancelación" debidos a la diferencia en la definición hacen que el resultado pierda sentido.

- La definición de la "derivada compleja" también converge *cuadráticamente*; a diferencia de las dos definiciones anteriores *no* exhibe problemas al considerar valores de $h$ muy pequeños.

Los puntos anteriores muestran que, la manera de implementar un algoritmo (concreto) cualquiera en la computadora es importante respecto a cuestiones de la convergencia y del manejo de errores numéricos. En este sentido, la "derivada compleja" da el resultado que más se acerca al exacto, incluso para valores de $h \sim 10.^{-16}$.

La pregunta es si podemos ir más allá y obtener el valor exacto, esto es, el que más se acerca al valor obtenido con números reales excepto por cuestiones de redondeo.

## Diferenciación automática (o algorítmica)

Citando a [wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation): ``Automatic differentiation (AD), also called algorithmic differentiation or computational differentiation, is a set of techniques to numerically evaluate the derivative of a function specified by a computer program.''

Diferenciación automática **no es** diferenciación numérica. Está basada en cálculos numéricos (evaluación de funciones por la computadora), pero **no** usa ninguna de las definiciones por diferencias finitas, como las que vimos la clase anterior.

Para ilustrar su funcionamiento, empezaremos recordando cómo funcionan los números complejos: $z = a + \mathrm{i} b$.

Uno puede definir todas las operaciones aritméticas de la manera natural (a partir de los números reales), manteniendo las expresiones con $\mathrm{i}$ factorizada; en el caso de la multiplicación y división, simplemente recordamos que $\mathrm{i}^2=-1$.

Por ejemplo, tenemos que, para $z = a + \mathrm{i} b$ y $w = c + \mathrm{i} d$
\begin{eqnarray*}
z + w & = & (a + \mathrm{i} b) + (c + \mathrm{i} d) = a + c + \mathrm{i}(b + d),\\
z \cdot w & = & (a + \mathrm{i} b)\cdot (c + \mathrm{i} d) \\
  & = & ac + \mathrm{i} (ad+bc) + \mathrm{i}^2 b d
 = ac - b d + \mathrm{i} (ad+bc).
\end{eqnarray*}

Por último, vale también la pena recordar que, $\mathbb{C}$ es *isomorfo* a $\mathbb{R}^2$, esto es, hay un mapeo biyectivo de $ z \leftrightarrow (a, b)$.

Volviendo a la cuestión de la diferenciación automática, la idea es introducir un par ordenado donde la primer
componente es el valor de una función $f(x)$ evaluada en $x_0$, y la segunda es el valor de su derivada evaluada
en el mismo punto. Esto es, definimos a los *duales* como:

\begin{equation}
\vec{f}(x_0) = \big( f_0, f'_0\big) = f_0 + \hat{\mathbf{h}}\, f'_0,
\end{equation}

donde $f_0 = f(x_0)$ y $f'_0=\frac{d f}{d x}(x_0)$ y, en la segunda igualdad, $\hat{\mathbf{h}}$ sirve para indicar la segunda componente del par ordenado. (En un sentido que se precisará más adelante, $\hat{\mathbf{h}}$ se comporta de una manera semejante a $\mathrm{i}$ para los números complejos, distinguiéndose en $\hat{\mathbf{h}}^2$.)

En particular, para la función constante $f(x)=c$ se cumple que el dual asociado es $\vec{c}(x_0)=(c,0)$, y para la función identidad $f(x)=x$ obtenemos $\vec{x}(x_0)=(x_0,1)$. Vale la pena aquí enfatizar que la función identidad es precisamente la variable independiente, respecto a la que se está derivando.

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina los duales, donde el parámetro debe ser un subtipo de `Real` (ver la siguiente celda). La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

In [1]:
"""Definición de los duales, donde 'fun' es la variable de la funcion
evaluada en x_0, y 'der' es la variable de la derivada en x_0
...
"""
type Dual{T<:Real}
    fun :: T
    der :: T    
end

In [2]:
# Definan un método que permita la promoción automática cuando no son del mismo tipo
# Aqui se define un método que garantiza que el dual de un número cumple lo requerido

import Base.convert
import Base.promote_rule
import Base.+

Dual(a,b) = Dual(promote(a,b)...)
Dual(a) = Dual(a, zero(a))

convert{T<:Real}(::Type{Dual{T}}, a::T) = Dual(a)
convert{T<:Real, S<:Real}(::Type{Dual{T}}, a::S) = Dual(convert(T,a))

promote_rule{T<:Real, S<:Real}(::Type{Dual{T}}, ::Type{S}) = 
    Dual{(promote_type)(T,S)}

promote_rule (generic function with 125 methods)

In [3]:
# Aqui se define la función `xdual`, que se usará para identificar la variable independiente
# La función depende de x_0, y debe regresar el Dual apropiado a la variable independiente
function xdual(x0)
    Dual(x0,1)
end

xdual (generic function with 1 method)

In [4]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test

# Aqui vienen varios tests de cada una de las funciones que implementaron
# Para checar la construcción de duales, hagánlo verificando cada uno de los campos internos
    @test Dual(5.,5).der == 5.
    @test Dual(3).fun == 3 && Dual(3).der == 0
    @test Dual(3.).fun == 3. && Dual(3.).der == 0
    d2 = Dual(7)
    @test d2.fun == 7 && d2.der == 0
    #Correcto. No marca error

## Aritmética de duales

De la definición, y recordando el significado de cada una de las componentes del par doble, tenemos que las operaciones aritméticas quedan definidas por:

\begin{eqnarray}
    \vec{u} \pm \vec{w} &=& \big( u_0 \pm w_0,\, u'_0\pm w'_0 \big),\\
    \vec{u} \times \vec{w} &=& \big( u_0 \cdot w_0,\, u_0 w'_0 +  w_0 u'_0 \big),\\
    \frac{\vec{u}}{\vec{w}} &=& \big( \frac{u_0}{w_0},\, \frac{ u'_0 - (u_0/w_0)w'_0}{w_0}\big),\\
    {\vec{u}}^\alpha &=& \big( u_0^\alpha,\, \alpha u_0^{\alpha-1} u'_0 \big).\\
\end{eqnarray}    

Además, están los operadores unitarios, que satisfacen:
\begin{equation}
    \pm \vec{u} = \big(\pm u_0, \pm u'_0 \big).
\end{equation}    

Noten que las dos relaciones anteriores son compatibles con $\hat{\mathbf{h}}^2=0$.

### Ejercicio

Implementen *todas* las operaciones aritméticas. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`). Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). Implementen también la comparación entre duales (`==`). Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [5]:
import Base: +, -, *, /, ^, ==

# Aqui se implementan los métodos necesarios para cada función

+(a::Dual, b::Dual) = Dual(a.fun + b.fun, a.der + b.der) 
+(a::Real, b::Dual) = Dual(a + b.fun, b.der)
+(a::Dual, b::Real) = Dual(a.fun + b, a.der)

-(a::Dual, b::Dual) = Dual(a.fun - b.fun, a.der - b.der) 
-(a::Real, b::Dual) = Dual(a - b.fun, b.der)
-(a::Dual, b::Real) = Dual(a.fun - b, a.der)

*(a::Dual, b::Dual) = Dual(a.fun * b.fun, a.fun*b.der + b.fun*a.der) 
*(a::Real, b::Dual) = Dual(a * b.fun, a*b.der)
*(a::Dual, b::Real) = Dual(a.fun * b, b*a.der)

/(a::Dual, b::Dual) = Dual(a.fun / b.fun, (a.der - ((a.fun/b.fun)*b.der)) / b.fun)
/(a::Real, b::Dual) = Dual(a / b.fun, ((a/b.fun)*b.der) / b.fun) 
/(a::Dual, b::Real) = Dual(a.fun / b, a.der / b)

^(a::Dual, c::Integer) = Dual(a.fun^c, (c*(a.fun^(c-1))) * a.der)
^(a::Dual, c::Real) = Dual(a.fun^c, (c*(a.fun^(c-1))) * a.der)

redondear(a::Dual) = Dual(round(a.fun), round(a.der))

convertir(a::Dual) = Dual(Float64(a.fun), round(a.der))

==(a::Dual, b::Dual) = a.fun==b.fun && a.der==b.der 

== (generic function with 110 methods)

In [6]:
# Aqui se incluyen las pruebas necesarias
@test Dual(2,2)+2 == Dual(4,2)
@test 2 + Dual(2,2) == Dual(4,2)
@test 2*Dual(2,0) == Dual(2,0)*2
@test 1/Dual(2,0) == Dual(1/2,0)
@test Dual(1,0)^2 == Dual(1,0)
d1 = Dual(2.0,3.0)
d2 = Dual(2.1,3.0)

Dual{Float64}(2.1,3.0)

In [7]:
@test d1 == d2 #Correcto, da error

LoadError: LoadError: test failed: Dual{Float64}(2.0,3.0) == Dual{Float64}(2.1,3.0)
 in expression: d1 == d2
while loading In[7], in expression starting on line 1

In [8]:
@test d1 + d2 == Dual(4, 6) #Correcto, da error

LoadError: LoadError: test failed: Dual{Float64}(4.1,6.0) == Dual{Int64}(4,6)
 in expression: d1 + d2 == Dual(4,6)
while loading In[8], in expression starting on line 1

In [9]:
@test d1 + d2 == Dual(4.1, 6)
#Correcto. No marca error

In [10]:
@test d1 + 2.0 == Dual(4.0, 3.0)
#Correcto. No marca error

In [11]:
@test 2.0 + d1 == Dual(4.0, 3.0)
#Correcto. No marca error

In [12]:
@test 2.0 + d1 == Dual(4.3, 3.0)
#Correcto. Marca error

LoadError: LoadError: test failed: Dual{Float64}(4.0,3.0) == Dual{Float64}(4.3,3.0)
 in expression: 2.0 + d1 == Dual(4.3,3.0)
while loading In[12], in expression starting on line 1

In [13]:
@test d1 - d2 == Dual(-0.1, 0)
#Incorrecto. No debería marcar error...

LoadError: LoadError: test failed: Dual{Float64}(-0.10000000000000009,0.0) == Dual{Float64}(-0.1,0.0)
 in expression: d1 - d2 == Dual(-0.1,0)
while loading In[13], in expression starting on line 1

In [14]:
#Haciendo lo siguiente ya no marca el error, es correcto el test
#res = d1-d2
#print(res)
#res = Float64(round(res*1000))/1000
pres = Dual(1000, 1000)
res = d1 - d2
#println(convertir(pres))
@test convertir(redondear((res*pres)))/pres == Dual(-0.1, 0)
#test = 2.0 - 2.1
#println(test)
#println(Int32(round(test*1000))/1000)

In [15]:
#Ahora ya es correcto el test

pres = Dual(1000, 1000)
res = d1 - d2
@test convertir(redondear((res*pres)))/pres == Dual(-0.1, 0)


In [16]:
#Este test es correcto, marca error
pres = Dual(1000, 1000)
res = d1 - d2

@test convertir(redondear((res*pres)))/pres == Dual(-0.1, 0.1)

LoadError: LoadError: test failed: Dual{Float64}(-0.1,0.0) == Dual{Float64}(-0.1,0.1)
 in expression: convertir(redondear(res * pres)) / pres == Dual(-0.1,0.1)
while loading In[16], in expression starting on line 5

In [17]:
@test d1*d2 == Dual(4.2, 12.3)
#Correcto. No da error

In [18]:
@test d1^2 == Dual(4, 24)
#Marca error, es correcto

LoadError: LoadError: test failed: Dual{Float64}(4.0,12.0) == Dual{Int64}(4,24)
 in expression: d1 ^ 2 == Dual(4,24)
while loading In[18], in expression starting on line 1

In [19]:
@test d1^2 == Dual(4, 12)
#Test correcto

Ahora probaremos la infraestructura que han desarrollado hasta ahora. En particular, usaremos la misma función que se
utilizón en la clase anterior:

\begin{equation}
    f_\textrm{test}(x) = 3x^3 - 2,
\end{equation}

y la idea es calcular la derivada en $x_0=1$.

In [20]:
f_test(x) = 3*x^3 - 2

f_test (generic function with 1 method)

In [21]:
f_test(xdual(1))

Dual{Int64}(1,9)

In [22]:
# Del resultado anterior, obtengan el valor de la derivada
xdual(1),xdual(2)

(Dual{Int64}(1,1),Dual{Int64}(2,1))

In [23]:
f_test(xdual(2))

Dual{Int64}(22,36)

In [24]:
f_test(xdual(3))

Dual{Int64}(79,81)

Consideremos otra función racional:

$$
g_\textrm{test}(x) = (3x^2-8x+5)/(7x^3-1)
$$

cuya derivada queremos calcular en $x_0=1$. Según [WolframAlpha](http://www.wolframalpha.com/input/?i=D%5B+%283x%5E2-8x%2B5%29%2F%287x%5E3-1%29+%2C+x+%5D+%2F.+x-%3E+1), el resultado exacto es $-1/3$.


In [25]:
gtest(x) = (3x^2-8x+5)/(7x^3-1)

gtest (generic function with 1 method)

In [26]:
gtest(xdual(1))

Dual{Float64}(0.0,-0.3333333333333333)

El resultado anterior es la representación en números de punto flotante de $-1/3$. La pregunta es si podemos obtener
el resultado exacto.

In [27]:
# Evalúen `gtest` de manera que el resultado sea el exacto
gtest(xdual(1//1))

Dual{Rational{Int64}}(0//1,-1//3)

Para entender cómo funciona esto, evaluaremos explícitamente y paso a paso $g_\textrm{test}(x)$ en $\vec{x_0} = (1,1)$:

\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = \frac{\vec{3}\cdot{\vec{x_0}}^2-\vec{8}\cdot\vec{x_0}+\vec{5}}{\vec{7}\cdot{\vec{x_0}}^3-\vec{1}}
\end{equation}


\begin{eqnarray}
\vec{g}_\textrm{test}(\vec{x_0}) & = & 
\frac{(3,0)\cdot{(1,1)}^2-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot{(1,1)}^3-(1,0)}\\
& = & \frac{(3,0)\cdot(1,2)-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot(1,3)-(1,0)}
\end{eqnarray}


\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = 
\frac{ (3,6)-(8,1)+(5,0)}{(7,21)-(1,0)} = \frac{ \Big(3-8+5, 6-8\Big)}{\Big( 7-1,21\Big)} = 
\frac{ \Big(0, -2\Big)}{\Big( 6,21\Big)} 
\end{equation}

\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = \Big(\frac{0}{6}, \frac{(-2)-21\cdot(0/6)}{6} \Big) = 
\Big(0, -\frac{1}{3}\Big). \Box
\end{equation}


## Modulos y "runtests.jl" en julia

### Ejercicio

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo coyo nombre será "AutomDiff.jl" y cuya estructura es:

```julia
# Aqui viene una explicación de lo que se hace en el módulo, los autores y la fecha

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^
    
    export Dual, xdual
    
    # Aqui viene TODO el código que implementaron

end
```

Todas las pruebas deberán ser incluídas en un archivo llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos


```

Estos dos archivos deben incluirlos en el envío de su tarea.